In [3]:
path = "PRIMNET/FINGER"
import os
os.chdir("..")

In [4]:
# %%
from pathlib import Path
import sys

try: 
    get_ipython().__class__.__name__
    BASEDIR = Path().absolute()
except: BASEDIR = Path(__file__).parent

sys.path.append(str(BASEDIR))
import torch
from torch import nn
import argparse

import numpy as np
from utils import dataloader
from utils.initalize import INITALZE_EVEN_JOINTS
from utils.update import  update_primnet, update_fc_primnet, update_pcc_primnet


import os
import random
from pathlib import Path
import wandb
import time
import json

from utils.tools import set_seed, set_wandb, print_log_dict, prefix_dict, average_dict
from utils.path_handler import JUPYTER, RUN, DEBUG, get_BASERDIR
from utils.args import read_ARGS
from utils.logger import CSVLogger,ask_and_make_folder
from utils.tools import cast_numpy

from configs.template import PRIMNET_ARGS_TEMPLATE, FC_PRIMNET_ARGS_TEMPLATE, PCC_PRIMNET_ARGS_TEMPLATE
from model.PRIMNET import PRIMNET
from model.FC_PRIMNET import FC_PRIMNET
from typing import Union

from utils.dataloader import get_dataset, Sampler

In [5]:
BASEDIR, RUNMODE = get_BASERDIR(os.getcwd())
args = read_ARGS((BASEDIR/'results'/path/"args.py").absolute())
args

Running on Jupyter...


ARGS(MODEL='PRIMNET', EVEN_JOINTS=True, WANDB=True, pname='PRIMNET_v2.1', runname='FINGER', DATASET='FINGER', TPOSE=((0, 0, 0.12),), LOAD_WEIGHTPATH=None, SAVE_PERIOD=1, TEST_PERIOD=1, EVEN_JOINT=True, p_offset_std=0.1, rpy_offset_std=0.01, axis_std=0.1, OUTPUT_NORMALIZE=False, seed=0, hdim=(16, 16), motor_embed_dim=4, lr=0.0015, lrd=0.95, wd=0.0, w_vec=0.001, epochs=500, focus_ratio=0.0, data_ratio=1.0, n_workers=2, batch_size=64, joint_seqs=('F', 'R', 'P', 'R', 'R', 'P', 'R'), marker_num=1, motor_dim=4)

In [6]:
model = PRIMNET(args=args).to(args.device)

model.load_state_dict(torch.load(BASEDIR/'results'/path/"weights/epoch_500.pth"))

<All keys matched successfully>

In [7]:
def forward_motor(model:PRIMNET, motor_control):     
    motor_control = model.normalize(motor_control)
    
    # Forward
    act_embeds = model.ACT_EMBED.layers(motor_control)
    q_values = model.FK_LAYER.forward_q(act_embeds)
    joint_se3 = model.FK_LAYER.forward_kinematics(q_values)
    
    return model.t2p(joint_se3, OUTPUT_NORMALIZE=False)



## Kinematic gradient Check

In [8]:
motor_control = torch.ones(1,args.motor_dim).to(args.device)

with torch.no_grad():
    act_embeds = model.ACT_EMBED.layers(motor_control)
    q_values = model.FK_LAYER.forward_q(act_embeds)

q_values = q_values.detach()
q_values.requires_grad_(True)


tensor([[ 0.0000, -0.0052, -0.0211,  0.0104, -0.0003, -0.0076, -0.0046]],
       requires_grad=True)

In [9]:
from torch import autograd, Tensor

def forward_kinematics(q_values:Tensor, idx=-1):
    assert q_values.ndim == 2
    assert q_values.shape[0] == 1
    
    joint_se3 = model.FK_LAYER.forward_kinematics(q_values)
    joint_position = model.t2p(joint_se3, OUTPUT_NORMALIZE=False)

    # aux_joints = len(args.joint_seqs)//args.marker_num

    # marker_position = []
    # for i in range(len(args.joint_seqs)):
    #     if (i+1)%aux_joints == 0:
    #         joint_position_ = joint_position[:,i]
    #         marker_position.append(joint_position_[0])

    # return torch.stack(marker_position, dim=0)[prim_idxs]
    return joint_position[0,idx,:,0]

EE_pos = forward_kinematics(q_values, idx=-1)

EE_pos.shape, q_values.shape

(torch.Size([3]), torch.Size([1, 7]))

In [10]:
from torch.autograd.functional import jacobian
from functools import partial


jac_auto = jacobian(partial(forward_kinematics, idx=6), q_values)
jac_auto = jac_auto[:,0,:]
jac_auto.shape, jac_auto

(torch.Size([3, 7]),
 tensor([[ 0.0000,  1.5545,  0.4118,  2.1619, -0.1977, -1.0993,  0.0000],
         [ 0.0000,  1.0748,  1.3707,  1.0633,  1.6178, -2.5206,  0.0000],
         [ 0.0000,  0.2390,  0.1161,  0.2985,  0.2746,  0.1201,  0.0000]]))

In [11]:
motor_control = model.normalize(motor_control)

# Forward
act_embeds = model.ACT_EMBED.layers(motor_control)
q_values = model.FK_LAYER.forward_q(act_embeds)
joint_se3 = model.FK_LAYER.forward_kinematics(q_values)

joint_position =  model.t2p(joint_se3, OUTPUT_NORMALIZE=False)[0,:,:,0]

EE_pos = joint_position[-1]

EE_pos.shape

torch.Size([3])

In [12]:
from utils.pyart import t2r

joint_rotation = t2r(joint_se3[0])
joint_rotation.shape

torch.Size([7, 3, 3])

In [13]:
from model.PRIMNET import Fjoint, Tjoint, Rjoint, Pjoint

jac_explicit = torch.zeros(3, len(model.FK_LAYER.joints))

for idx,joint in enumerate(model.FK_LAYER.joints):
    if isinstance(joint, Fjoint):
        continue
    elif isinstance(joint, Tjoint):
        continue
    elif isinstance(joint, Rjoint):
        pos_diff = EE_pos - joint_position[idx]
        jac_explicit[:, idx] = torch.cross(joint_rotation[idx] @ joint.axis.data[:,0], pos_diff)
        # print('here')
    elif isinstance(joint, Pjoint):
        pos_diff = EE_pos - joint_position[idx]
        jac_explicit[:,idx] = joint_rotation[idx] @ joint.axis.data[:,0]

jac_explicit

tensor([[ 0.0000,  1.5523,  0.4091,  2.1767, -0.1897, -1.1088, -0.0000],
        [ 0.0000,  1.0695,  1.3713,  1.0501,  1.6179, -2.5162,  0.0000],
        [ 0.0000,  0.2355,  0.1179,  0.2864,  0.2686,  0.1241,  0.0000]],
       grad_fn=<CopySlices>)

## Time Check

In [14]:
import time
n = 100

start_time = time.time()
for _ in range(n):
    jac_auto = jacobian(partial(forward_kinematics, idx=6), q_values)
    jac_auto = jac_auto[:,0,:]
end_time = time.time()

autograd_time = end_time-start_time
print("[AutoGrad]:{:2f}".format(autograd_time))

start_time = time.time()
for _ in range(n):
    jac_explicit = torch.zeros(3, len(model.FK_LAYER.joints))

    with torch.no_grad():
        act_embeds = model.ACT_EMBED.layers(motor_control)
        q_values = model.FK_LAYER.forward_q(act_embeds)
        joint_se3 = model.FK_LAYER.forward_kinematics(q_values)

        joint_position =  model.t2p(joint_se3, OUTPUT_NORMALIZE=False)[0,:,:,0]

        EE_pos = joint_position[-1]

    for idx,joint in enumerate(model.FK_LAYER.joints):
        if isinstance(joint, Fjoint):
            continue
        elif isinstance(joint, Tjoint):
            continue
        elif isinstance(joint, Rjoint):
            pos_diff = EE_pos - joint_position[idx]
            jac_explicit[:, idx] = torch.cross(joint_rotation[idx] @ joint.axis.data[:,0], pos_diff)
            # print('here')
        elif isinstance(joint, Pjoint):
            pos_diff = EE_pos - joint_position[idx]
            jac_explicit[:,idx] = joint_rotation[idx] @ joint.axis.data[:,0]
end_time = time.time()
explicit_time = end_time-start_time
print("[Explicit]:{:2f}".format(explicit_time))

print("Result: [Explicit] is faster than [AutoGrad] by {:2f} times".format(autograd_time/explicit_time))



[AutoGrad]:0.788793
[Explicit]:0.346618
Result: [Explicit] is faster than [AutoGrad] by 2.275682 times


In [15]:
n = 100

start_time = time.time()
for _ in range(n):
    forward_kinematics(q_values, idx=6)
end_time = time.time()

with_grad_time = end_time-start_time

start_time = time.time()
for _ in range(n):
    with torch.no_grad():
        forward_kinematics(q_values, idx=6)
end_time = time.time()
without_grad_time = end_time-start_time

print("Result: [without_grad] is faster than [with_grad] by {:2f} times".format(with_grad_time/without_grad_time))

Result: [without_grad] is faster than [with_grad] by 1.373967 times
